In [1]:
import os
from monai.data import load_decathlon_datalist, Dataset
from utils import load_itk
import numpy as np

os.listdir('/data/')

['HC-clean-nodules-coord-preview',
 'HC1',
 'HC1-lung-segment',
 'HC1-lung-segment-preview',
 'HC1-lung-segment-preview1',
 'HC1-nodules-coordinates-preview',
 'LUNA-sample',
 'LUNA16_datasplit',
 'LUNA16_Images',
 'LUNA16_Images_resample',
 'MSD',
 'MSD-clean-nodules-coord-preview',
 'MSD-lung-segment',
 'MSD-lung-segment-preview',
 'MSD-nodules-coordinates-preview',
 'output']

In [2]:
base_dir="/data/LUNA16_Images_resample/"
downloaded_datasplit_dir='/data/LUNA16_datasplit'
data_list_file_path = os.path.join(downloaded_datasplit_dir, "mhd_original/dataset_fold0.json")

In [3]:
inference_data = load_decathlon_datalist(
    data_list_file_path,
    is_segmentation=True,
    data_list_key="training",
    base_dir=base_dir
)
inference_ds = Dataset(
    data=inference_data,
    # transform=inference_transforms
)

In [4]:
inference_data

[{'box': [[100.0021827,
    22.00611876,
    -147.0757546,
    8.871621958,
    8.871621958,
    8.871621958],
   [-55.31574942,
    4.042374673,
    -154.7552924,
    5.970538617,
    5.970538617,
    5.970538617],
   [-25.40860122,
    136.715217,
    -181.6943198,
    11.08601237,
    11.08601237,
    11.08601237],
   [127.2115066,
    44.79847847,
    -164.7894133,
    7.72427841,
    7.72427841,
    7.72427841],
   [120.2208995,
    50.312055,
    -132.1372108,
    5.251592099,
    5.251592099,
    5.251592099],
   [101.082909,
    -3.953008029,
    -180.4023007,
    9.404294354,
    9.404294354,
    9.404294354]],
  'image': '/data/LUNA16_Images_resample/1.3.6.1.4.1.14519.5.2.1.6279.6001.250397690690072950000431855143.mhd',
  'label': [0, 0, 0, 0, 0, 0]},
 {'box': [[102.2377292,
    65.32842988,
    -240.6955559,
    18.210145,
    18.210145,
    18.210145],
   [22.53616818,
    34.72145001,
    -117.4375995,
    10.33971307,
    10.33971307,
    10.33971307],
   [60.18947724,
  

### testing voxel to world coordinate conversion

In [5]:
image, origin, spacing = load_itk(
    '/data/LUNA16_Images/' \
    '1.3.6.1.4.1.14519.5.2.1.6279.6001.109002525524522225658609808059.mhd'
)
print(image.shape, origin, spacing)

(161, 512, 512) [-194.       -108.300003 -187.699997] [1.25       0.54882801 0.54882801]


In [6]:
# exam, origin, spacing = load_itk(
#     '/data/LUNA16_Images_resample/' \
#     '1.3.6.1.4.1.14519.5.2.1.6279.6001.108197895896446896160048741492/' \
#     '1.3.6.1.4.1.14519.5.2.1.6279.6001.108197895896446896160048741492.nii.gz'
# )

In [58]:
# origin_min = np.array((106, 342.25, 31.453125))
origin_min = np.array((413.75, 273.25, 62.90625))
# origin_max = np.array((114.75, 351, 34.0625))
origin_max = np.array((438.5, 298, 73.8125))
# origin = np.array(( -182.5, -190, -313.75))
origin = np.array((-187.699997, -108.300003, -194))
# original_spacing = np.array((0.74218798, 0.74218798, 2.5)) # original spacing
original_spacing = np.array((0.54882801, 0.54882801, 1.25)) # original spacing
# original_spacing = np.array((0.703125, 0.703125, 1.25))

world_coord_min = origin + origin_min*original_spacing
world_coord_max = origin + origin_max*original_spacing
world_coord_diff = origin_min - origin_max
print(world_coord_min)
print(world_coord_max)
print(world_coord_diff)

world_coord_cent = (world_coord_min + world_coord_max)/2
world_coord_diam = world_coord_min - world_coord_max

print(world_coord_cent)
print(world_coord_diam)

[  39.37759214   41.66725073 -115.3671875 ]
[  52.96108539   55.25074398 -101.734375  ]
[-24.75    -24.75    -10.90625]
[  46.16933876   48.45899736 -108.55078125]
[-13.58349325 -13.58349325 -13.6328125 ]


In [ ]:
[  39.37759214   41.66725073 -115.3671875 ]
[  52.96108539   55.25074398 -101.734375  ]
[-24.75    -24.75    -10.90625]
[  46.16933876   48.45899736 -108.55078125]
[-13.58349325 -13.58349325 -13.6328125 ]

### testing world to voxel

In [48]:
world_coord_cent = np.array((46.18853869, 48.40280596, -108.5786324))
world_coord_diam = np.array((13.59647134, 13.59647134, 13.59647134))

world_coord_max = (2*world_coord_cent - world_coord_diam)/2
world_coord_min = world_coord_diam + world_coord_max

print(world_coord_max)
print(world_coord_min)

origin = np.array((-187.699997, -108.300003, -194))
original_spacing = np.array((0.54882801, 0.54882801, 1.25)) # original spacing

origin_min = (world_coord_min - origin)/original_spacing
origin_max = (world_coord_max - origin)/original_spacing
print(origin_min)
print(origin_max)

[  39.39030302   41.60457029 -115.37686807]
[  52.98677436   55.20104163 -101.78039673]
[438.54680697 297.90943912  73.77568262]
[413.77316005 273.13579219  62.89850554]


could not find the difference, both using voxel to world and world to voxel! Though they are so close, think about it later! (the solution may be in LIDC-IDRI documentation!)

In [44]:
413.78498351/413.75

1.0000845522900303

In [45]:
413.75/413.78498351

0.9999154548584551

In [49]:
297.90943912/298

0.9996961044295302

In [50]:
298/297.90943912

1.0003039879510616

In [35]:
image_shape = (512, 512, 161)
new_spacing = np.array((0.703125, 0.703125, 1.25))
resize_factor = original_spacing / new_spacing
new_real_shape = image_shape * resize_factor
new_shape = np.round(new_real_shape)
real_resize_factor = new_shape / image_shape
new_spacing = new_spacing / real_resize_factor
print(new_spacing)
print(real_resize_factor)
print(new_shape)

[0.9  0.9  1.25]
[0.78125 0.78125 1.     ]
[400. 400. 161.]


In [9]:
0.703125/0.74218798

0.9473678083549669

In [10]:
6.440878725/-6.49414482

-0.9917978276622418

In [30]:
6.49414482*0.99

6.4292033718

In [16]:
-6.49414482/6.440878725

-1.0082700043385773

In [14]:
-13.58349325/13.59647134

-0.9990454810166944

In [31]:
13.59647134*0.99

13.4605066266

In [15]:
13.59647134/-13.58349325

-1.0009554309602944

In [11]:
world_coord_diam*0.9473678083549669

array([-6.15234375, -6.15234375, -6.18009469])